# Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [31]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [32]:
from utils import get_api_key
openai_api_key = get_api_key('OPENAI_API_KEY')
gemini_api_key = get_api_key('GEMINI_API_KEY')

# print(gemini_api_key)


In [33]:
from crewai import Agent, Task, Crew, LLM

In [34]:
# llm = LLM(
#     api_key=openai_api_key,
#     model='gpt-3.5-turbo'
# )
llm = LLM(
    api_key=gemini_api_key,
    model='gemini/gemini-1.5-flash'
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [35]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

In [36]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [37]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [38]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [39]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [40]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [41]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [42]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan: Understanding the Rise of Artificial Intelligence**

**1. Prioritized Trends, Key Players, and News:**

* **Trends:** Generative AI (e.g., DALL-E 2, Midjourney, Stable Diffusion, ChatGPT), AI in Healthcare (diagnosis, drug discovery), AI-powered Cybersecurity, Responsible AI development and ethics,  Edge AI (AI processing at the device level),  AI for Sustainability.
* **Key Players:** OpenAI, Google (DeepMind), Microsoft, Amazon (AWS), Meta, Nvidia, IBM.  Noteworthy startups in specific AI niches should also be mentioned depending on the chosen focus.
* **News:**  Recent b

In [47]:
result

CrewOutput(raw="# Understanding the Rise of Artificial Intelligence: A Revolution in Progress\n\n**Introduction**\n\nArtificial intelligence (AI) is rapidly transforming our world.  From creating art to diagnosing diseases and predicting cybersecurity threats, AI's capabilities are expanding at an unprecedented rate. At its core, AI involves creating machines capable of performing tasks that typically require human intelligence, encompassing machine learning (systems learning from data) and deep learning (a subset using artificial neural networks). This blog post offers an overview of current AI trends, key players, and their impact, aiming to provide a clear and accessible understanding of this transformative technology.  Navigating the complexities of the 21st century requires a grasp of AI's evolution and implications.\n\n\n**Key Points: Navigating the AI Landscape**\n\n**A. The Generative AI Revolution: Creativity Unleashed**\n\nGenerative AI represents a major advancement, enablin

In [48]:
from IPython.display import Markdown, display
display(Markdown(result.raw))



# Understanding the Rise of Artificial Intelligence: A Revolution in Progress

**Introduction**

Artificial intelligence (AI) is rapidly transforming our world.  From creating art to diagnosing diseases and predicting cybersecurity threats, AI's capabilities are expanding at an unprecedented rate. At its core, AI involves creating machines capable of performing tasks that typically require human intelligence, encompassing machine learning (systems learning from data) and deep learning (a subset using artificial neural networks). This blog post offers an overview of current AI trends, key players, and their impact, aiming to provide a clear and accessible understanding of this transformative technology.  Navigating the complexities of the 21st century requires a grasp of AI's evolution and implications.


**Key Points: Navigating the AI Landscape**

**A. The Generative AI Revolution: Creativity Unleashed**

Generative AI represents a major advancement, enabling machines to produce new content—text, images, video, and audio.  Tools like ChatGPT (known for conversational abilities) and DALL-E 2 and Stable Diffusion (generating visuals from text) showcase this potential. Applications range from automated content creation to artistic expression. However, potential misuse, such as creating deepfakes or spreading misinformation, necessitates careful consideration of ethical implications, including bias and copyright. Responsible development and deployment are crucial to maximize benefits and mitigate risks.


**B. AI's Impact Across Industries: Transforming the World**

AI is revolutionizing various sectors. In healthcare, AI-powered diagnostics improve accuracy and speed, accelerating drug discovery and enabling personalized medicine.  Cybersecurity benefits from AI's enhanced threat detection and prevention.  Finance utilizes AI for algorithmic trading, fraud detection, and risk management. Manufacturing employs AI for automation and predictive maintenance, boosting efficiency. These are just a few examples of AI's wide-ranging transformative potential.


**C. Key Players Shaping the AI Landscape: Giants and Startups**

Tech giants like OpenAI, Google (DeepMind), Microsoft, Amazon (AWS), Meta, and Nvidia are heavily invested in AI research and development, driving advancements across subfields. Their contributions, in foundational models and industry applications, are substantial.  A vibrant startup ecosystem also contributes, focusing on niche areas and innovative solutions. This dynamic interplay of large corporations and agile startups fuels competition and innovation, accelerating AI's progress.


**D. Ethical Considerations and Responsible AI: Navigating the Moral Maze**

The rapid advancement of AI necessitates addressing ethical concerns. Algorithmic bias, potentially exacerbating societal inequalities, is a major worry.  Data privacy requires robust protection measures.  Job displacement due to automation and the potential for malicious use of AI are also significant issues.  A multifaceted approach—ethical guidelines, transparency, and collaboration among researchers, policymakers, and industry leaders—is essential.  Responsible development and deployment are paramount.


**E. Future Trends and Predictions: A Glimpse into Tomorrow**

Future AI trends include edge AI (processing data directly on devices), enhancing efficiency and privacy, and AI for Sustainability, addressing climate change and resource management.  Advancements in natural language processing, computer vision, and robotics will lead to even more sophisticated AI systems. While predicting the precise trajectory is difficult, AI's profound impact on our world is undeniable.


**Conclusion**

Artificial intelligence is no longer a futuristic concept; it's a transformative technology shaping our present and future. Understanding its capabilities, limitations, and ethical implications is crucial for everyone.  Responsible development and deployment will determine whether AI serves as a force for progress or a source of potential harm.  Continued dialogue and collaboration are vital to navigate this transformative era.


**Call to Action**

Share your thoughts and predictions for the future of AI in the comments.  What ethical concerns do you find most pressing? For further reading, explore resources such as the Partnership on AI and the IEEE's work on ethical guidelines. Let's engage in a constructive discussion about this powerful technology.